# Bokeh

In [1]:
# !pip install bokeh

In [2]:
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

In [5]:
elements.head()

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,EA,standard state,bonding type,melting point,boiling point,density,metal,year discovered,group,period
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.20,37.0,NaN,120.0,...,-73.0,gas,diatomic,14.0,20.0,0.00009,nonmetal,1766,1,1
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,0.0,gas,atomic,NaN,4.0,0.00000,noble gas,1868,18,1
2,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.98,134.0,76 (+1),182.0,...,-60.0,solid,metallic,454.0,1615.0,0.54000,alkali metal,1817,1,2
3,4,Be,Beryllium,9.012182,#C2FF00,[He] 2s2,1.57,90.0,45 (+2),NaN,...,0.0,solid,metallic,1560.0,2743.0,1.85000,alkaline earth metal,1798,2,2
4,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.04,82.0,27 (+3),NaN,...,-27.0,solid,covalent network,2348.0,4273.0,2.46000,metalloid,1807,13,2


In [6]:
df = (
    elements.copy()
    .astype({"atomic mass": "str", "group": "str"})
    .assign(
        period=lambda d: [periods[x - 1] for x in d.period]
    )
    .query("""
        group != '-' \
        and not symbol.str.contains('L[ru]')
    """)
)

df.head()

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,EA,standard state,bonding type,melting point,boiling point,density,metal,year discovered,group,period
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.20,37.0,NaN,120.0,...,-73.0,gas,diatomic,14.0,20.0,0.00009,nonmetal,1766,1,I
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,0.0,gas,atomic,NaN,4.0,0.00000,noble gas,1868,18,I
2,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.98,134.0,76 (+1),182.0,...,-60.0,solid,metallic,454.0,1615.0,0.54000,alkali metal,1817,1,II
3,4,Be,Beryllium,9.012182,#C2FF00,[He] 2s2,1.57,90.0,45 (+2),NaN,...,0.0,solid,metallic,1560.0,2743.0,1.85000,alkaline earth metal,1798,2,II
4,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.04,82.0,27 (+3),NaN,...,-27.0,solid,covalent network,2348.0,4273.0,2.46000,metalloid,1807,13,II


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 117
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   atomic number             88 non-null     int64  
 1   symbol                    88 non-null     object 
 2   name                      88 non-null     object 
 3   atomic mass               88 non-null     object 
 4   CPK                       88 non-null     object 
 5   electronic configuration  88 non-null     object 
 6   electronegativity         67 non-null     float64
 7   atomic radius             69 non-null     float64
 8   ion radius                67 non-null     object 
 9   van der Waals radius      37 non-null     float64
 10  IE-1                      73 non-null     float64
 11  EA                        70 non-null     float64
 12  standard state            73 non-null     object 
 13  bonding type              73 non-null     object 
 14  melting poi

In [8]:
color_mapper = {
    "alkali metal": "#a6cee3",
    "alkaline earth metal": "#1f78b4",
    "metal": "#d93b43",
    "halogen": "#999d9a",
    "metalloid": "#e08d49",
    "noble gas": "#eaeaea",
    "nonmetal": "#f1d4Af",
    "transition metal": "#599d7A",
}

source = ColumnDataSource(df)
p = figure(
    width=900,
    height=500,
    title="Periodic table (omitting LA and AC series)",
    x_range=groups,
    y_range=list(reversed(periods)),
    toolbar_location=None,
    tools="hover",
)

In [9]:
p.rect(
    x="group",
    y="period",
    width=0.95,
    height=0.95,
    source=source,
    fill_alpha=0.6,
    legend_field="metal",
    color=factor_cmap("metal", palette=list(color_mapper.values()), factors=list(color_mapper.keys())),
)
show(p)

In [10]:
class Property(dict):
    def __init__(
        self,
        y,
        x=None,
        source=source,
        text_align="left",
        text_baseline="middle",
        text_font_size="16px",
        text_font_style="normal",
    ) -> None:
        super().__init__(
            y=y,
            x=x or dodge("group", -0.4, range=p.x_range),
            source=source,
            text_align=text_align,
            text_baseline=text_baseline,
            text_font_size={"value": text_font_size},
            text_font_style=text_font_style,
        )

props = {
    "atomic number": Property(
        y=dodge("period", 0.3, range=p.y_range), text_font_size="11px"
    ),
    "symbol": Property(y="period", text_font_style="bold"),
    "atomic mass": Property(y=dodge("period", -0.2, range=p.y_range), text_font_size="7px"),
    "name": Property(y=dodge("period", -0.35, range=p.y_range), text_font_size="7px"),
}

for name, prop in props.items():
    p.text(text=name, **prop)


show(p)

In [11]:
p.text(
    x=["3", "3"],
    y=["VI", "VII"],
    text=["LA", "AC"],
    text_align="center",
    text_baseline="middle",
)

show(p)

In [12]:
p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location = "top_center"

show(p)

In [13]:
p.hover.tooltips = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass", "@{atomic mass}"),
    ("Type", "@metal"),
    ("Electronic configuration", "@{electronic configuration}"),
]

show(p)